In [51]:
import numpy as np
import random
import math
import random
import telebot

In [54]:
bot = telebot.TeleBot('6570394944:AAGoXsiJY5A9LH-YUY66hevkUTLEc-YvT1A')

# Define a function to handle incoming messages
@bot.message_handler(func=lambda message: True)
def handle_message(message):
  # Send the data to yourself
  bot.send_message('b3hzadsh', message.text)

# Start the bot
bot.polling()

In [ ]:
def distance(p1, p2):
  """
  محاسبه فاصله اقلیدسی دو نقطه

  Args:
    p1: نقطه اول
    p2: نقطه دوم

  Returns:
    فاصله اقلیدسی بین دو نقطه
  """

  x1, y1 = p1
  x2, y2 = p2

  dx = x1 - x2
  dy = y1 - y2

  return math.sqrt(dx**2 + dy**2)

In [3]:
def calculate_distance(cities, solution):
    total_distance = 0
    for i in range(len(solution) - 1):
        city1 = cities[solution[i]]
        city2 = cities[solution[i + 1]]
        distance = ((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)**0.5
        total_distance += distance
    return total_distance


In [5]:
def randomGeneration():
  return np.random.permutation(100)

In [6]:
def read_city_data(filename):
    city_data = []
    with open(filename, 'r') as f:
        for line in f:
            city, lat, lon = line.strip().split(' ')
            city_data.append([city, int(lat), int(lon)])
            # check city is good verification 
            # id , lat (x), long(y) in array
    return city_data

In [7]:
city_data = read_city_data('city-cor.txt')
print(len(city_data))
# for city, lat, lon in city_data:
#     print(f"City: {city}, Latitude: {lat}, Longitude: {lon}")

100


In [8]:
def read_city_answer(filename):
    city_answer = []
    with open(filename, 'r') as f:
        for line in f:
            city_index = int(line)
            city_answer.append(city_index)
            # id , lat (x), long(y) in tupple
    return city_answer

In [9]:
city_ans = read_city_answer('answer.txt')
city_ans

[1,
 47,
 93,
 28,
 67,
 58,
 61,
 51,
 87,
 25,
 81,
 69,
 64,
 40,
 54,
 2,
 44,
 50,
 73,
 68,
 85,
 82,
 95,
 13,
 76,
 33,
 37,
 5,
 52,
 78,
 96,
 39,
 30,
 48,
 100,
 41,
 71,
 14,
 3,
 43,
 46,
 29,
 34,
 83,
 55,
 7,
 9,
 57,
 20,
 12,
 27,
 86,
 35,
 62,
 60,
 77,
 23,
 98,
 91,
 45,
 32,
 11,
 15,
 17,
 59,
 74,
 21,
 72,
 10,
 84,
 36,
 99,
 38,
 24,
 18,
 79,
 53,
 88,
 16,
 94,
 22,
 70,
 66,
 26,
 65,
 4,
 97,
 56,
 80,
 31,
 89,
 42,
 8,
 92,
 75,
 19,
 90,
 49,
 6,
 63]

In [10]:
print(city_data[0])

['1', 1380, 939]


In [11]:
def generate_random_chromosomes(population_size):
    chromosomes = []
    for _ in range(population_size):
        chromosome = randomGeneration()
        # random.shuffle(chromosome)
        chromosomes.append(chromosome)
    return chromosomes

In [12]:
def calculate_fitness(chromosome, cities=city_data):
    total_distance = 0
    if len(chromosome) != 100 :
      raise ValueError
    for i in range(100):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]
        
        city1 = cities[city1_index-1]
        city2 = cities[city2_index-1]

        latitude_diff = city1[1] - city2[1]
        longitude_diff = city1[2] - city2[2]

        distance = math.sqrt(latitude_diff**2 + longitude_diff**2)
        total_distance += distance
    return int(total_distance // 1)

In [13]:
calculate_fitness(city_ans)

21285

In [14]:
def calculate_fitness_np(chromosome, cities=city_data):
    if len(chromosome) != 100 :
        print('wrong chromosome length')
        raise ValueError

    total_distance = 0
    for i in range(100):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]
        
        city1 = np.array(cities[city1_index-1][1:])
        city2 = np.array(cities[city2_index-1][1:])
        
        distance = np.linalg.norm(city1 - city2)
        total_distance += distance

    return int(total_distance // 1)

In [15]:
calculate_fitness_np(city_ans)

21285

In [16]:
population = generate_random_chromosomes(500)
print(population[0])

[20 25 80 92 58 65  9 26 12 78 81 85 21  0 24 53 14 27 99 72 34 88 96 82
 44 95 38 54 47  7 66 60 98 61 68 62 70  3 39 59 63 97 31 42 90 57 55 77
 22 23 43 46 19 51 41 49  1 48 11 83  8 93 67 69 94 17  6 37 52  2 73 36
 87 28 79 91 29 74 33  4 32 13 86 10 56 71 84 89 76 40 45 16 18  5 50 35
 75 15 30 64]


In [83]:
for chro in population :
  fitforchro = calculate_fitness(chro)
  if fitforchro < 21285:
    print(fitforchro)

In [84]:
# fitness_values = []
# for chromosome in population :
#     fitness_value = calculate_fitness(chromosome, cities = city_data)
#     fitness_values.append(fitness_value)
# fitness_values

In [17]:
fitness_values = np.array([calculate_fitness(chromosome, cities=city_data) for chromosome in population])
print(fitness_values)

[176125 172586 170029 170277 172140 173101 177409 159300 173763 171427
 162268 175763 163218 166727 159037 167156 175002 172522 145345 177612
 174636 169747 172600 178839 168446 165836 154752 163354 179593 164295
 171233 172411 174506 178384 175202 158693 185812 174340 155032 176109
 173396 175888 184422 157314 165306 163926 173529 164885 168231 158724
 166592 175417 162539 160057 145900 170774 167751 173422 176975 175571
 165690 165539 185742 178724 189307 171515 165741 160845 174196 178541
 176903 157086 174536 172402 174531 188585 166985 181763 169649 179136
 166380 178478 182489 190403 167642 160365 180845 167860 180491 169401
 157316 167231 175828 174490 186255 177145 177441 169025 169894 186740
 173274 185168 181054 167531 161907 174223 169862 155599 171922 170660
 177736 171361 175857 182934 165311 172766 167314 182107 164406 161910
 185545 157998 175346 184313 197871 170372 172497 166982 169035 174795
 176052 172424 173954 171940 179906 173590 178943 162065 181272 165249
 18047

In [18]:
def permutation_parent_selection(population, fitness_values, selection_method="roulette_wheel"):
  """Selects parents for a genetic algorithm using permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    selection_method: The parent selection method to use. Supported methods are
      "roulette_wheel" and "tournament".

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  if selection_method == "roulette_wheel":
    return roulette_wheel_permutation_parent_selection(population, fitness_values)
  elif selection_method == "tournament":
    return tournament_permutation_parent_selection(population, fitness_values)
  else:
    raise ValueError("Unsupported parent selection method: {}".format(selection_method))


def roulette_wheel_permutation_parent_selection(population, fitness_values):
  """Selects parents using roulette wheel selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  total_fitness = np.sum(fitness_values)
  fitness_probabilities = fitness_values / total_fitness
  cumulative_probabilities = np.cumsum(fitness_probabilities)

  random_value = random.random()
  parent1_index = 0
  for i, probability in enumerate(cumulative_probabilities):
    if random_value < probability:
      parent1 = population[i]
      break

  random_value = random.random()
  parent2_index = 0
  for i, probability in enumerate(cumulative_probabilities):
    if random_value < probability:
      parent2 = population[i]
      break

  return parent1, parent2


def tournament_permutation_parent_selection(population, fitness_values, tournament_size=2):
  """Selects parents using tournament selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    tournament_size: The size of the tournament.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  parent1 = None
  parent2 = None

  for _ in range(tournament_size):
    random_index = random.randint(0, len(population) - 1)

    if parent1 is None or fitness_values[random_index] > fitness_values[parent1]:
      parent1 = population[random_index]

    if parent2 is None or fitness_values[random_index] > fitness_values[parent2]:
      parent2 = population[random_index]

  return parent1, parent2


In [19]:
roulette_wheel_permutation_parent_selection(population=population, fitness_values=fitness_values)

(array([40, 14, 69, 22, 83, 18, 34, 84, 56, 51, 13, 58, 61, 74, 37, 12, 17,
        55, 45, 31, 53, 29, 39, 92, 48, 60,  2, 25, 52, 41, 42, 46, 65, 49,
        88,  5, 85, 24, 15, 71, 68, 73, 16, 64, 62, 89, 30, 94,  6, 72, 80,
         8, 43, 91, 81,  7, 97, 90, 50, 28, 19,  1, 36, 96, 66, 95, 32, 87,
        38, 47, 99, 93, 10, 54, 26, 70, 63, 98, 44, 11, 35, 33,  3, 57, 79,
        21,  4,  9, 86, 67, 77, 82, 75, 20, 76,  0, 27, 59, 23, 78]),
 array([ 8, 17, 93, 23,  5, 48, 39, 86, 72,  7, 18,  9, 91,  3, 37, 43, 64,
        33,  4, 94, 42, 24, 20, 58,  0, 29, 67,  2, 95, 21, 49, 75, 41, 10,
        82, 55, 15, 80, 66, 44, 96, 89, 59, 62, 26,  1, 27, 68, 12, 45, 40,
        28, 73, 83,  6, 52, 97, 53, 50, 11, 98, 56, 99, 88, 14, 60, 92, 38,
        46, 87, 30, 34, 70, 22, 79, 61, 85, 90, 51, 63, 16, 71, 54, 81, 19,
        74, 78, 69, 84, 13, 36, 25, 57, 76, 47, 65, 32, 77, 31, 35]))

In [21]:
def PMX_crossover(parent1, parent2, seed):
    # key = random.PRNGKey(72)
    '''
    parent1 and parent2 are 1D np.array
    '''
    rng = np.random.default_rng(seed=seed)

    cutoff_1, cutoff_2 = np.sort(rng.choice(np.arange(len(parent1)+1), size=2, replace=False))

    def PMX_one_offspring(p1, p2):
        offspring = np.zeros(len(p1), dtype=p1.dtype)

        # Copy the mapping section (middle) from parent1
        offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

        # copy the rest from parent2 (provided it's not already there
        for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
            candidate = p2[i]
            while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                # print(f"Candidate {candidate} not valid in position {i}") # DEBUGONLY
                candidate = p2[np.where(p1 == candidate)[0][0]]
            offspring[i] = candidate
        return offspring

    offspring1 = PMX_one_offspring(parent1, parent2)
    offspring2 = PMX_one_offspring(parent2, parent1)

    return offspring1, offspring2
parent1 = np.random.permutation(10)
parent2 = np.random.permutation(10)
# Print the parents
print("Parent 1:", parent1)
print("Parent 2:", parent2)
# Apply PMX and print the offspring
offspring1, offspring2 = PMX_crossover(parent1, parent2,313)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Parent 1: [2 1 6 0 9 3 4 8 7 5]
Parent 2: [9 8 7 6 0 4 3 1 5 2]
Offspring 1: [6 8 7 0 9 3 4 1 5 2]
Offspring 2: [2 1 9 6 0 4 3 8 7 5]


In [22]:
offspring1, offspring2 = PMX_crossover(population[0], population[1],313)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [71 87 38  2 45 86 80 13 82 34 16 21 89 53 18  7 56 28 78 58 73 81 10 95
 64 72 36 17 88 94 84 60 98 61 68 62 70  3 39 59 63 97 31 42 90 57 55 77
 22 23 43 46 19 51 41 49  1 48 11 83  8 93 67 85 14 69 54  4 74 27 26 52
 47 29 79  6 20  9 37 33 76 25 92 12 44  5 91 96 99 40 65 15  0 32 50 75
 35 24 66 30]
Offspring 2: [20 25 80 92  3 65  9 60 42 78 81 85 21  0 24 53 14 27  1 49 34 88 96 61
 57 95 38 54 47  7 66 26 97 82 45 59 75 58 30  2 99  6 28 12 40 44 48 94
  8 76 23 13 71 32 86 72 93 90 33 56 98 63 52 69 77 17 22 37 67 62 73 36
 87 31 79 91 29 74 11  4 51 46 41 10 83 19 84 89 43 55 68 16 18  5 50 35
 70 15 39 64]


In [23]:
firstPermutation = np.random.permutation(100) # generate a random permutation of integers from 0 to 99
print(firstPermutation.shape)
secondPermutation = np.random.permutation(100) # generate another random permutation of integers from 0 to 99
PMX_crossover(firstPermutation, secondPermutation,313) # apply the PMX

(100,)


(array([91, 50, 68, 80, 13, 84, 70, 33,  9, 30, 66,  6, 16, 61,  8, 31, 51,
        76,  0, 53, 95, 38, 26, 74, 93, 25, 78, 27, 67, 21, 15, 69, 29, 86,
        90, 88, 99, 77, 39, 63, 92, 17, 35, 98, 87, 71, 45, 89, 59, 49,  4,
        64,  3,  5, 14, 37, 23, 62, 46, 73, 97, 85, 11, 56, 22, 48, 60,  1,
        79, 32, 65,  7, 41, 55, 81, 52, 18, 43, 44, 19,  2, 40, 42, 57, 75,
        24, 12, 82, 47, 83, 72, 94, 54, 28, 10, 20, 96, 34, 58, 36]),
 array([71, 56, 84, 54, 65, 87, 41,  4, 32, 97, 72,  6, 35, 63, 60, 85,  3,
        23, 92,  0, 46, 17, 81,  2, 57, 61,  1, 44, 20, 74, 99, 51, 69, 26,
        95, 90, 12, 86, 40,  9, 82, 49, 75, 59, 77, 11, 68, 21, 96, 50, 89,
        42, 31, 16, 58, 93, 28, 79, 15, 24, 10, 73, 43, 36, 76, 78, 30, 19,
        98, 70, 13, 29, 14, 52, 27, 34, 47,  5, 37, 33, 45, 64, 83, 62, 66,
         7, 18, 91, 53, 22,  8, 38, 67, 80, 94, 39, 55, 48, 25, 88]))

In [95]:
def cxPartialyMatched(ind1, ind2):
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2
offspring1, offspring2 = cxPartialyMatched(population[0], population[1])
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [ 5 72  7 76 94 26 97 60 98 40  1 32  2 50 28 20 90  8 22 46 10 78 88 66
  6 15 82 73 75 25  3 83 91 63 65 33 87  0 49 92 17 16 52 30 56 59 79 36
 14 45 95 84 43 74 19 71 68 64 54 21 80 29 96 85 37 23 69 51 62 53 93 11
 61 81 41  9 58 89 42 48 70 38 39 13 18 77 44 24 57 12 27 47 55  4 34 86
 67 31 35 99]
Offspring 2: [55 28 80  1  3 15 36 86 34 74 93 26 96 51 61 83 94 68 45 60 77 17 12 46
 42 85 31 16  4 88 70  6 11  5 35 27 56 53 67 69 52 10 48 21 19 65 62 90
 20 22 72 38 50 87 40 78  9 37 64 30 25 32 79 76 81 92  0 63 43 41 91 59
 14 73 44 84 13 24 66 99 57 75  8 23 54 95 71 89 82 33 39 97 49  2 58 18
  7 29 98 47]


In [96]:
def erx_crossover(parent1, parent2):
    """Performs Edge Recombination Crossover (ERX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    offspring = []
    visited = set()

    # Start from a random gene in parent1
    gene = random.choice(parent1)
    offspring.append(gene)
    visited.add(gene)

    while len(offspring) < len(parent1):
        # Find the next gene to add
        next_gene = None

        # Check if the next gene in parent2 is not in the visited set
        next_gene_in_parent2 = parent2[(parent1.index(gene) + 1) % len(parent1)]
        if next_gene_in_parent2 not in visited:
            next_gene = next_gene_in_parent2

        # Check if the next gene in parent1 is not in the visited set
        if next_gene is None:
            next_gene_in_parent1 = parent1[(parent2.index(gene) + 1) % len(parent2)]
            if next_gene_in_parent1 not in visited:
                next_gene = next_gene_in_parent1

        # If no valid next gene found, break
        if next_gene is None:
            break

        # Add the next gene to the offspring and mark it as visited
        offspring.append(next_gene)
        visited.add(next_gene)

        # Update the current gene
        gene = next_gene

    return offspring

In [97]:
def ox_crossover(parent1, parent2):
    """Performs Order Crossover (OX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    crossover_points = random.sample(range(1, len(parent1)), 2)
    slice_start, slice_end = crossover_points[0], crossover_points[1]

    slice1 = parent1[slice_start:slice_end]
    remaining_genes1 = set(parent1) - set(slice1)

    mapping = {}
    for i in range(slice_start, slice_end):
        gene1 = parent1[i]
        gene2 = parent2[i]

        if gene1 in remaining_genes1:
            mapping[gene1] = gene2
            remaining_genes1.remove(gene1)

    offspring = []
    for gene in parent2:
        if gene in mapping:
            offspring.append(mapping[gene])
        elif gene not in slice1:
            offspring.append(gene)

    for gene in slice1:
        offspring.append(gene)

    return offspring

In [98]:
def cycle_crossover(parent1, parent2):
    """
    Performs Cycle Crossover (CX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    cycle_start = random.randrange(len(parent1))
    cycle_genes1 = set(parent1[cycle_start:])
    cycle_genes2 = set(parent2[cycle_start:])

    offspring = list(parent1[:cycle_start])
    for gene in parent2:
        if gene in cycle_genes1:
            offspring.append(gene)
            cycle_genes1.remove(gene)

            if not cycle_genes1:
                break
        else:
            cycle_genes2.add(gene)

    for gene in cycle_genes2:
        offspring.append(gene)

    return offspring

In [44]:
def swap_mutation(chromosome):
    i, j = random.sample(range(len(chromosome)), 2)
    chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

In [45]:
original_chromosome = [1, 2, 3, 4, 5]
swap_mutation(original_chromosome)

[4, 2, 3, 1, 5]

In [100]:
def scramble_mutation(chromosome):
    """Performs Scramble mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    random.shuffle(sub_chromosome)
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [101]:
def insert_mutation(chromosome):
    """Performs Insert mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    i = random.randrange(len(chromosome))
    j = random.randrange(len(chromosome))

    if i != j:
        gene = chromosome.pop(i)
        chromosome.insert(j, gene)

    return chromosome

In [102]:
def inversion_mutation(chromosome):
    """Performs Inversion mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    sub_chromosome.reverse()
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [103]:
def inversion_mutation(permutation):
  # randomly select two positions in the permutation
  i, j = np.random.choice(len(permutation), size=2, replace=False)
  # make sure i is smaller than j
  if i > j:
    i, j = j, i
  # invert the segment between i and j
  permutation[i:j+1] = permutation[i:j+1][::-1]
  return permutation


In [24]:
def inversion_mutation(chromosome):
  """Performs inversion mutation on a permutation chromosome.

  Args:
    chromosome: A NumPy array of city indices in permutation representation.

  Returns:
    A NumPy array of city indices in permutation representation representing the mutated chromosome.
  """

  # Select two random indices in the chromosome.
  index1 = np.random.randint(0, len(chromosome))
  index2 = np.random.randint(0, len(chromosome))

  # Ensure that the two indices are different.
  while index1 == index2:
    index2 = np.random.randint(0, len(chromosome))

  # Reverse the order of the genes between the two indices.
  # print(type(chromosome[index1:index2 + 1]))
  chromosome[index1:index2 + 1] = chromosome[index1:index2 + 1][::-1]

  return chromosome


# Fake data
chromosome = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Mutate the chromosome
mutated_chromosome = inversion_mutation(chromosome)

# Print the mutated chromosome
print(mutated_chromosome)


[ 6  5  4  3  2  1  7  8  9 10]


In [105]:
# import unittest
# import numpy as np

# def inversion_mutation(permutation):
#     print('permutation is : ')
#     print(permutation)
#     # randomly select two positions in the permutation
#     i, j = np.random.choice(len(permutation), size=2, replace=False)
#     # make sure i is smaller than j
#     if i > j:
#         i, j = j, i
#     # invert the segment between i and j
#     permutation[i:j+1] = permutation[i:j+1][::-1]
#     return permutation

# class TestInversionMutation(unittest.TestCase):
#     def test_inversion_mutation(self):
#         # Test case 1: permutation with length 5
#         permutation = [1, 2, 3, 4, 5]
#         mutated_permutation = inversion_mutation(permutation)
#         self.assertEqual(len(mutated_permutation), len(permutation))
#         self.assertNotEqual(mutated_permutation, permutation)

#         # Test case 2: permutation with length 10
#         permutation = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
#         mutated_permutation = inversion_mutation(permutation)
#         self.assertEqual(len(mutated_permutation), len(permutation))
#         self.assertNotEqual(mutated_permutation, permutation)

# if __name__ == '__main__':
#     unittest.main()


In [25]:
PMX_crossover(population[0], population[1],313)

(array([71, 87, 38,  2, 45, 86, 80, 13, 82, 34, 16, 21, 89, 53, 18,  7, 56,
        28, 78, 58, 73, 81, 10, 95, 64, 72, 36, 17, 88, 94, 84, 60, 98, 61,
        68, 62, 70,  3, 39, 59, 63, 97, 31, 42, 90, 57, 55, 77, 22, 23, 43,
        46, 19, 51, 41, 49,  1, 48, 11, 83,  8, 93, 67, 85, 14, 69, 54,  4,
        74, 27, 26, 52, 47, 29, 79,  6, 20,  9, 37, 33, 76, 25, 92, 12, 44,
         5, 91, 96, 99, 40, 65, 15,  0, 32, 50, 75, 35, 24, 66, 30]),
 array([20, 25, 80, 92,  3, 65,  9, 60, 42, 78, 81, 85, 21,  0, 24, 53, 14,
        27,  1, 49, 34, 88, 96, 61, 57, 95, 38, 54, 47,  7, 66, 26, 97, 82,
        45, 59, 75, 58, 30,  2, 99,  6, 28, 12, 40, 44, 48, 94,  8, 76, 23,
        13, 71, 32, 86, 72, 93, 90, 33, 56, 98, 63, 52, 69, 77, 17, 22, 37,
        67, 62, 73, 36, 87, 31, 79, 91, 29, 74, 11,  4, 51, 46, 41, 10, 83,
        19, 84, 89, 43, 55, 68, 16, 18,  5, 50, 35, 70, 15, 39, 64]))

In [36]:
def compare_max_values(array1, array2):
  """
  check if first array has bigger max   
  """
  min_value1 = np.min(array1)
  min_value2 = np.min(array2)

  if min_value1 < min_value2:
    return False
  else:
    return True

# Example usage
array1 = np.array([1, 2, 3, 4, 5])
array2 = np.array([6, 7, 8, 9, 10])

compare_max_values(array1, array2)

False

In [47]:
# def roulette_wheel_selection(population, fitness_values):
#     total_fitness = np.sum(fitness_values)
#     fitness_probabilities = fitness_values / total_fitness
#     cumulative_probabilities = np.cumsum(fitness_probabilities)

#     random_index = random.random()
#     for i, probability in enumerate(cumulative_probabilities):
#         # print(f'i is {i}')
#         if random_index < probability:
#             return population[i]
# def roulette_wheel_selection(population, fitness_values):
#     total_fitness = np.sum(fitness_values)
#     fitness_probabilities = fitness_values / total_fitness
#     cumulative_probabilities = np.cumsum(fitness_probabilities)

#     random_value = random.random()
#     for i, probability in enumerate(cumulative_probabilities):
#         if random_value < probability:
#             selected_chromosome = population[i]
#             break

#     # Shuffle the selected chromosome to ensure no bias in selecting the next parent
#     # np.random.shuffle(selected_chromosome)

#     return selected_chromosome

# def tournament_selection(population , cities = city_data, tournament_size = 5):
#     # Create a tournament
#     tournament = random.sample(population, tournament_size)
#     # Select the best individual among the tournament
#     best_individual = min(tournament, key=lambda x: calculate_distance(cities, x))
#     return best_individual

def genetic_algorithm(population_size=500, crossover_rate=0.7, mutation_rate=0.1, max_generations=700,cities = city_data):
    chromosomes = generate_random_chromosomes(500)
    print(len(chromosomes))
    fitness_values = []
    for chromosome in chromosomes :
        fitness_value = calculate_fitness_np(chromosome, cities)
        fitness_values.append(fitness_value)

    generation_without_improvement = 0

    for generation in range(max_generations):
        # Select parents using roulette wheel selection
        parents = []
        for i in range(population_size):
            parent1 , parent2 = permutation_parent_selection(chromosomes, fitness_values)
            parents.append((parent1, parent2))

        offsprings = []
        for parent1, parent2 in parents:
            # print(parent1)
            # break
            if random.random() < crossover_rate:
                offspring1 ,offspring2 = PMX_crossover(parent1, parent2,313)
            else:
                offspring1 = parent1
                offspring2 = parent2

            offsprings.append(offspring1)
            offsprings.append(offspring2)

        # Apply mutation
        for offspring in offsprings:
            if random.random() < mutation_rate:
                offspring = swap_mutation(offspring)
                # offspring = inversion_mutation(offspring)

        # Evaluate offspring fitness
        offspring_fitness_values = []
        for offspring in offsprings :
            # calculate_fitness(offspring, cities)
            offspring_fitness_value = calculate_fitness_np(offspring)
            # if offspring_fitness_value < 21285 :# todo remove this if for times reducing
            #     print(offspring_fitness_value)
            offspring_fitness_values.append(offspring_fitness_value)

        # Combine parents and offspring to form a new population
        new_population = np.empty_like(chromosomes)
        new_fitness_values = np.empty_like(fitness_values)

        for i in range(population_size):
            if offspring_fitness_values[i] < fitness_values[i]:
                new_population[i] = offsprings[i]
                new_fitness_values[i] = offspring_fitness_values[i]
            else:
                new_population[i] = chromosomes[i]
                new_fitness_values[i] = fitness_values[i]

        chromosomes = new_population
        fitness_values = new_fitness_values

        # Check for stopping condition
        if compare_max_values(offspring_fitness_values,fitness_values):
            generation_without_improvement += 1

            if generation_without_improvement >= 20:
                break
        else:
            generation_without_improvement = 0

    best_chromosome = chromosomes[fitness_values.argmax()]
    best_fitness = np.min(fitness_values)

    return best_chromosome, best_fitness


In [56]:
result = genetic_algorithm()

500


In [55]:
print(result)

(array([31, 65,  8,  4, 26, 56, 89, 67, 77, 60, 23, 20, 47, 91, 61, 51, 58,
       42, 80, 70, 94, 38, 36, 84, 30, 14, 48,  3, 46, 66, 75, 97, 19,  6,
       63, 35, 12, 55, 29, 34, 83,  1, 90, 79, 88, 22, 16, 53, 49, 43, 71,
       41,  0, 52, 78,  5, 37, 92, 10, 18, 24, 99, 59, 11, 98, 85, 68, 25,
       15, 17, 74, 21, 72, 32, 45, 62, 86, 27, 57,  7,  9, 87, 28, 93, 81,
       39, 96, 50, 44, 73, 69, 64, 40, 54,  2, 95, 76, 33, 13, 82]), 53902)
